<a href="https://colab.research.google.com/github/choycoy/Final/blob/main/recommendation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
q_df = pd.read_csv("/content/drive/My Drive/final/questions.csv",sep=",")

In [ ]:
q_df['question_id'] = q_df['question_id'].str.replace('q', '')
q_df

In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__1.csv",sep=",")
df2 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__2.csv",sep=",")
df3 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__3.csv",sep=",")
df4 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__4.csv",sep=",")
df5 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__5.csv",sep=",")
df6 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__6.csv",sep=",")
df7 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__7.csv",sep=",")
df8 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__8.csv",sep=",")
df9 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__9.csv",sep=",")
df10 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__10.csv",sep=",")
df11 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__11.csv",sep=",")
df12 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__12.csv",sep=",")
df13 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__13.csv",sep=",")
df14 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__14.csv",sep=",")
df15 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__15.csv",sep=",")
df16 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__16.csv",sep=",")
df17 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__17.csv",sep=",")
df18 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__18.csv",sep=",")
df19 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__19.csv",sep=",")
df20 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__20.csv",sep=",")
df21 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__21.csv",sep=",")
df22 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__22.csv",sep=",")
df23 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__23.csv",sep=",")
df24 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__24.csv",sep=",")
df25 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__25.csv",sep=",")
df26 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__26.csv",sep=",")
df27 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__27.csv",sep=",")
df28 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__28.csv",sep=",")


df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8,df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20,df21, df22,df23, df24, df25, df26, df27, df28])

In [ ]:
# Convert the 'question_id' column in df1 and df2 to the same data type (e.g., int64)
df['question_id'] = df['question_id'].astype('int64')
q_df['question_id'] = q_df['question_id'].astype('int64')

# Merge the dataframes on 'question_id'
merged_df = pd.merge(df, q_df[['question_id', 'correct_answer']], on='question_id', how='left')

# Create the 'is_correct' column based on the comparison between 'user_answer' and 'correct_answer'
merged_df['is_correct'] = (merged_df['user_answer'] == merged_df['correct_answer']).astype(int)

# Now merged_df will have a new column 'is_correct' indicating if the user_answer is correct or not
print(merged_df)

In [ ]:
# Sort the dataframe by timestamp to ensure the data is in chronological order
merged_df.sort_values(by=['user_id', 'timestamp'], inplace=True)

# Calculate Correctness Rate and Interaction Count for each user
merged_df['correct_count'] = merged_df.groupby('user_id')['is_correct'].cumsum()  # Cumulative sum of correct answers for each user
print(merged_df['correct_count'])

merged_df['interaction_count'] = merged_df.groupby('user_id').cumcount() + 1  # Cumulative count of interactions for each user
print(merged_df['interaction_count'])

merged_df['correctness_rate'] = merged_df['correct_count'] / merged_df['interaction_count']  # Calculate the correctness rate for each interaction
merged_df['average_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('mean')

merged_df

In [ ]:
user_correctness = merged_df.groupby('user_id')['correctness_rate'].mean()
user_correctness

In [ ]:
user_elapsed_time = merged_df.groupby('user_id')['average_elapsed_time'].mean()
user_elapsed_time

In [ ]:
user_metrics_df = pd.DataFrame({'user_id': user_correctness.index,
                                'correctness_rate': user_correctness.values,
                                'user_elapsed_time': user_elapsed_time.values})

In [ ]:
# Calculate the 80th percentile (quantile) of 'elapsed_time_scaled'
top_20_percentile = user_metrics_df['user_elapsed_time'].quantile(0.3)
below_20_percentile = user_metrics_df['user_elapsed_time'].quantile(0.7)

high_achievers_threshold = 0.7
struggling_learners_threshold = 0.3

def segment_users(row):
    if row['correctness_rate'] >= high_achievers_threshold and row['user_elapsed_time'] <= top_20_percentile:
        return 'High Achievers'
    elif row['correctness_rate'] <= struggling_learners_threshold and row['user_elapsed_time'] >= below_20_percentile:
        return 'Struggling Learners'
    else:
        return 'Average Performers'

user_metrics_df['user_segment'] = user_metrics_df.apply(segment_users, axis=1)


In [ ]:
pip install surprise

In [ ]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(0, 1))
custom_dataset = Dataset.load_from_df(merged_df[['user_id', 'question_id', 'correctness_rate', 'average_elapsed_time']], reader)

In [ ]:
trainset, testset = train_test_split(custom_dataset, test_size=0.2)

In [ ]:
from surprise import NMF

# Choose the NMF algorithm
algo = NMF()

# Train the model on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

In [ ]:
from surprise import CoClustering

# Choose the CoClustering algorithm
algo = CoClustering()

# Train the model on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


In [ ]:
from surprise import SVD

# Choose the SVD algorithm
algo = SVD()

# Train the model on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

In [ ]:
user_id = 3
user_segment = user_metrics_df[user_metrics_df['user_id'] == user_id]['user_segment'].values[0]

if user_segment == 'High Achievers':
    # Recommend challenging questions
    recommended_questions = [(question_id, algo.predict(user_id, question_id).est) for question_id in range(trainset.n_items) if algo.predict(user_id, question_id).est <= 0.4]
elif user_segment == 'Struggling Learners':
    # Recommend moderately challenging questions
    recommended_questions = [(question_id, algo.predict(user_id, question_id).est) for question_id in range(trainset.n_items) if algo.predict(user_id, question_id).est >= 0.6]
else:
    # Recommend easier questions or remedial content
    recommended_questions = [(question_id, algo.predict(user_id, question_id).est) for question_id in range(trainset.n_items) if 0.4 < algo.predict(user_id, question_id).est < 0.6]

# Sort recommended_questions based on the prediction score in descending order
recommended_questions.sort(key=lambda x: x[1], reverse=True)

print(f"Top 10 Detailed Recommendations for user {user_id}:")
for i, (question_id, prediction_score) in enumerate(recommended_questions[:10], 1):
    print(f"{i}. Question ID: {question_id}, Prediction Score: {prediction_score}")
